In [1]:
import pandas as pd
import numpy as np

In [ ]:
def pad_sequence(seq, max_length=48, pad_char='-'):
    return seq.ljust(max_length, pad_char)
char_to_index = {
    'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4, 'Q': 5, 'E': 6, 'G': 7, 'H': 8, 'I': 9,
    'L': 10, 'K': 11, 'M': 12, 'F': 13, 'P': 14, 'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19,
    '-': 20  
}
def sequence_to_numbers(seq):
    return [char_to_index[char] for char in seq]
def numbers_to_sequence(seq):
    pass

df = pd.read_csv('lib/datasets/Protein_sequences/grampa.csv')
sequences = df['sequence']
seq = sequences[sequences.str.len().between(12, 48)]
seq = seq.drop_duplicates()

In [ ]:
df = pd.read_table('lib/datasets/Protein_sequences/stream')


In [ ]:
print(df)
df_seq = df['Sequence']

In [ ]:

seq = df_seq[df_seq.str.len().between(12, 48)]
seq = seq.drop_duplicates()

pad_seq = seq.apply(pad_sequence).reset_index(drop=True)
print(pad_seq)
num_seq = pad_seq.apply(sequence_to_numbers)
print(num_seq)
arr_num = np.stack(num_seq.values, 0)
print(arr_num, type(arr_num), arr_num.shape)


In [ ]:
df = pd.read_csv('lib/datasets/Protein_sequences/grampa.csv')
sequences = df['sequence']
seq = sequences[sequences.str.len().between(12, 48)]
seq = seq.drop_duplicates()
print(seq)
pad_seq = seq.apply(pad_sequence).reset_index(drop=True)
print(pad_seq)
num_seq = pad_seq.apply(sequence_to_numbers)
print(type(num_seq[0][0]))
arr_str = np.stack(pad_seq.values, 0)
print(arr_str, type(arr_str), arr_str.shape)
print(num_seq)
print(num_seq[2])

In [64]:
import torch
from lib.datasets.protein import IDX2CHAR
seq_arr = np.load('lib/datasets/Protein_sequences/grampa_numarr.npy')

print(seq_arr[0], type(seq_arr[0]))
load_tn = torch.from_numpy(seq_arr).to('cpu')

[ 7 10 14  1 11  9 10  4  0  9  0 11 11 11  7 11  4 11  7 14 10 11 10 19
  4 11  4 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20] <class 'numpy.ndarray'>


In [65]:
char_array = np.vectorize(IDX2CHAR.get)(seq_arr)
print(char_array)

[['G' 'L' 'P' ... '-' '-' '-']
 ['G' 'F' 'G' ... '-' '-' '-']
 ['L' 'V' 'K' ... '-' '-' '-']
 ...
 ['G' 'I' 'I' ... '-' '-' '-']
 ['K' 'L' 'W' ... '-' '-' '-']
 ['L' 'N' 'K' ... '-' '-' '-']]


In [17]:
print(load_tn, type(load_tn), load_tn.shape)
dataloader = torch.utils.data.DataLoader(load_tn,
    batch_size=3,
    shuffle=False)


tensor([[ 7, 10, 14,  ..., 20, 20, 20],
        [ 7, 13,  7,  ..., 20, 20, 20],
        [10, 19, 11,  ..., 20, 20, 20],
        ...,
        [ 7,  9,  9,  ..., 20, 20, 20],
        [11, 10, 17,  ..., 20, 20, 20],
        [10,  2, 11,  ..., 20, 20, 20]]) <class 'torch.Tensor'> torch.Size([5235, 48])


In [62]:
import lib.utils.utils as utils
from config.config_hollow_maze import get_config
from lib.models.models import UniformRate, UniformVariantRate
cfg = get_config()
cfg.data.S = 21
cfg.model.rate_const = 0.32
cfg.model.t_func = "sqrt_cos"
model = UniformRate(cfg, 'cpu')
device ='cpu'



for minibatch in dataloader: 
    B = minibatch.shape[0]
    ts = torch.rand((B,), device=device) * (1.0 - 0.01) + 0.01
    ts = torch.ones((B,), device=device) * 1
    print(ts)

    qt0 = model.transition(ts)  # (B, S, S)
    b = utils.expand_dims(
        torch.arange(B, device=device), (tuple(range(1, minibatch.dim())))
    )

    qt0 = qt0[b, minibatch.long()]
    print(qt0, )

    log_qt0 = torch.where(qt0 <= 0.0, -1e9, torch.log(qt0))

    xt = torch.distributions.categorical.Categorical(
        logits=log_qt0).sample()
    print("xt")
    print(xt)
    print("minibatch")
    print(minibatch)
    break
    

tensor([1., 1., 1.])
tensor([[[0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0476],
         [0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0476],
         [0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0476],
         ...,
         [0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0488],
         [0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0488],
         [0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0488]],

        [[0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0476],
         [0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0476],
         [0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0476],
         ...,
         [0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0488],
         [0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0488],
         [0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0488]],

        [[0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0476, 0.0476],
         [0.0476, 0.0476, 0.0476,  ..., 0.0476, 0.0488, 0.0476],
         [0.0476, 0.0476, 0.0476,  ..